In [45]:
import geopandas as gpd
import pandas as pd
import xarray as xr
from earthkit import transforms

from dhis2eo.data.cds import era5_land
from dhis2eo.data.worldpop import pop_total

In [ ]:
org_units = gpd.read_file('../guides/data/nepal-locations.geojson')

## Download and aggregate climate data

In [ ]:
start = "2022"
end = "2024"
bbox = org_units.total_bounds
dirname = '../guides/data/local'
prefix = 'era5_monthly_nepal'
variables = ["2m_temperature", "total_precipitation"]

files = era5_land.monthly.download(start=start, end=end, bbox=bbox, dirname=dirname, prefix=prefix, variables=variables)
files

INFO - 2026-02-16 14:28:08,697 - dhis2eo.data.cds.era5_land.monthly - File already downloaded: C:\Users\karimba\Documents\Github\climate-tools\docs\data\local\era5_monthly_nepal_2022-2024.nc


[WindowsPath('C:/Users/karimba/Documents/Github/climate-tools/docs/data/local/era5_monthly_nepal_2022-2024.nc')]

In [8]:
monthly_ds = xr.open_dataset(files[0])
monthly_ds = monthly_ds.drop_vars(['number', 'expver'])
monthly_ds

<xarray.Dataset> Size: 993kB
Dimensions:     (valid_time: 36, latitude: 42, longitude: 82)
Coordinates:
  * valid_time  (valid_time) datetime64[ns] 288B 2022-01-01 ... 2024-12-01
  * latitude    (latitude) float64 336B 30.45 30.35 30.25 ... 26.55 26.45 26.35
  * longitude   (longitude) float64 656B 80.06 80.16 80.26 ... 87.96 88.06 88.16
Data variables:
    t2m         (valid_time, latitude, longitude) float32 496kB ...
    tp          (valid_time, latitude, longitude) float32 496kB ...
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2026-02-16T13:25 GRIB to CDM+CF via cfgrib-0.9.1...

In [42]:
precip = monthly_ds['tp'] * 1000 * monthly_ds.valid_time.dt.days_in_month
precip.name = 'tp'
agg_precip = transforms.spatial.reduce(precip, org_units, mask_dim='id', how='mean')
agg_precip_df = agg_precip.to_dataframe().reset_index()
agg_precip_df.rename(columns={'id':'location', 'valid_time':'time_period'}, inplace=True)
agg_precip_df['time_period'] = agg_precip_df['time_period'].apply(lambda v: str(v)[:7])
agg_precip_df.sort_values(['location', 'time_period'], inplace=True)
agg_precip_df

,location,time_period,tp
684,A3R7UT64jHf,2022-01,17.197408
685,A3R7UT64jHf,2022-02,45.246871
686,A3R7UT64jHf,2022-03,2.248218
687,A3R7UT64jHf,2022-04,19.814122
688,A3R7UT64jHf,2022-05,72.042217
...,...,...,...
1687,zkFZKX5RX1A,2024-08,407.584344
1688,zkFZKX5RX1A,2024-09,351.301458
1689,zkFZKX5RX1A,2024-10,32.659165
1690,zkFZKX5RX1A,2024-11,26.415400


In [41]:
temp = monthly_ds['t2m'] - 273.15
temp.name = 't2m_c'
agg_temp = transforms.spatial.reduce(temp, org_units, mask_dim='id', how='mean')
agg_temp_df = agg_temp.to_dataframe().reset_index()
agg_temp_df.rename(columns={'id':'location', 'valid_time':'time_period'}, inplace=True)
agg_temp_df['time_period'] = agg_temp_df['time_period'].apply(lambda v: str(v)[:7])
agg_temp_df.sort_values(['location', 'time_period'], inplace=True)
agg_temp_df

,location,time_period,t2m_c
684,A3R7UT64jHf,2022-01,14.822235
685,A3R7UT64jHf,2022-02,16.140289
686,A3R7UT64jHf,2022-03,25.421051
687,A3R7UT64jHf,2022-04,29.116486
688,A3R7UT64jHf,2022-05,29.182892
...,...,...,...
1687,zkFZKX5RX1A,2024-08,13.836334
1688,zkFZKX5RX1A,2024-09,12.694440
1689,zkFZKX5RX1A,2024-10,9.214101
1690,zkFZKX5RX1A,2024-11,5.723183


## Download and aggregate population data

In [ ]:
start = "2022"
end = "2024"
country_code = 'NPL'
dirname = '../guides/data/local'
prefix = 'worldpop_yearly_nepal'

files = pop_total.yearly.download(start=start, end=end, country_code=country_code, dirname=dirname, prefix=prefix)
files

INFO - 2026-02-16 14:51:23,621 - dhis2eo.data.worldpop.pop_total.yearly - Year 2022
INFO - 2026-02-16 14:51:23,623 - dhis2eo.data.worldpop.pop_total.yearly - Reading 2022 -> https://data.worldpop.org/GIS/Population/Global_2015_2030/R2025A/2022/NPL/v1/100m/constrained/npl_pop_2022_CN_100m_R2025A_v1.tif
INFO - 2026-02-16 14:51:32,686 - dhis2eo.data.worldpop.pop_total.yearly - Year 2023
INFO - 2026-02-16 14:51:32,689 - dhis2eo.data.worldpop.pop_total.yearly - Reading 2023 -> https://data.worldpop.org/GIS/Population/Global_2015_2030/R2025A/2023/NPL/v1/100m/constrained/npl_pop_2023_CN_100m_R2025A_v1.tif
INFO - 2026-02-16 14:51:41,233 - dhis2eo.data.worldpop.pop_total.yearly - Year 2024
INFO - 2026-02-16 14:51:41,235 - dhis2eo.data.worldpop.pop_total.yearly - Reading 2024 -> https://data.worldpop.org/GIS/Population/Global_2015_2030/R2025A/2024/NPL/v1/100m/constrained/npl_pop_2024_CN_100m_R2025A_v1.tif


[WindowsPath('C:/Users/karimba/Documents/Github/climate-tools/docs/data/local/worldpop_yearly_nepal_2022.nc'),
 WindowsPath('C:/Users/karimba/Documents/Github/climate-tools/docs/data/local/worldpop_yearly_nepal_2023.nc'),
 WindowsPath('C:/Users/karimba/Documents/Github/climate-tools/docs/data/local/worldpop_yearly_nepal_2024.nc')]

In [48]:
yearly_ds = xr.open_mfdataset(files)
yearly_ds

C:\Users\karimba\AppData\Local\Temp\ipykernel_15504\3607737633.py:1: FutureWarning: In a future version of xarray the default value for data_vars will change from data_vars='all' to data_vars=None. This is likely to lead to different results when multiple datasets have matching variables with overlapping values. To opt in to new defaults and get rid of these warnings now use `set_options(use_new_combine_kwarg_defaults=True) or set data_vars explicitly.
  yearly_ds = xr.open_mfdataset(files)


<xarray.Dataset> Size: 1GB
Dimensions:      (time: 3, y: 4921, x: 9773)
Coordinates:
  * time         (time) datetime64[ns] 24B 2022-01-01 2023-01-01 2024-01-01
  * y            (y) float64 39kB 30.45 30.45 30.45 30.44 ... 26.35 26.35 26.35
  * x            (x) float64 78kB 80.06 80.06 80.06 80.06 ... 88.2 88.2 88.2
Data variables:
    spatial_ref  (time) int64 24B 0 0 0
    pop_total    (time, y, x) float64 1GB dask.array<chunksize=(1, 4921, 9773), meta=np.ndarray>

In [ ]:
pop = yearly_ds['pop_total']
agg_pop = transforms.spatial.reduce(pop, org_units, mask_dim='id', how='sum')

KeyError: 'time_period'

In [64]:
agg_pop_df = agg_pop.to_dataframe().reset_index()
agg_pop_df.rename(columns={'id':'location', 'time':'time_period', 'pop_total':'population'}, inplace=True)
agg_pop_df['time_period'] = agg_pop_df['time_period'].apply(lambda v: str(v)[:4])
agg_pop_df.sort_values(['location', 'time_period'], inplace=True)
agg_pop_df

,location,time_period,population
57,A3R7UT64jHf,2022,839709.512127
58,A3R7UT64jHf,2023,839700.646516
59,A3R7UT64jHf,2024,838533.146580
135,A3TeVhjjS2u,2022,253646.967689
136,A3TeVhjjS2u,2023,253644.290396
...,...,...,...
190,ycBkX3TlIrL,2023,259821.895553
191,ycBkX3TlIrL,2024,259460.529554
138,zkFZKX5RX1A,2022,58302.826951
139,zkFZKX5RX1A,2023,58302.211997


In [65]:
months = pd.DataFrame({'month': range(1, 12+1)})
agg_pop_df = agg_pop_df.merge(months, how='cross')
agg_pop_df['time_period'] = agg_pop_df['time_period'] + "-" + agg_pop_df['month'].astype(str).str.zfill(2)
agg_pop_df.drop(columns=['month'], inplace=True)
agg_pop_df

,location,time_period,population
0,A3R7UT64jHf,2022-01,839709.512127
1,A3R7UT64jHf,2022-02,839709.512127
2,A3R7UT64jHf,2022-03,839709.512127
3,A3R7UT64jHf,2022-04,839709.512127
4,A3R7UT64jHf,2022-05,839709.512127
...,...,...,...
2767,zkFZKX5RX1A,2024-08,58221.129061
2768,zkFZKX5RX1A,2024-09,58221.129061
2769,zkFZKX5RX1A,2024-10,58221.129061
2770,zkFZKX5RX1A,2024-11,58221.129061


## Export to csv files

In [ ]:
agg_precip_df.to_csv('../guides/data/nepal-era5-prcp-monthly-admin.csv', index=False)
agg_temp_df.to_csv('../guides/data/nepal-era5-t2m-monthly-admin.csv', index=False)
agg_pop_df.to_csv('../guides/data/nepal-worldpop-monthly-admin.csv', index=False)